In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the us-east-1 region. You will use the 811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [2]:
bucket_name = 'cloudprojectbanking' #Creating a bucket
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [3]:
#Loading data from S3 bucket to working instance
bucket='cloudprojectbanking'
data_key = 'UniversalBank 2.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
model_data = pd.read_csv(data_location)

In [4]:
#Splitting the data
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(3500, 14) (1500, 14)


In [5]:
train_data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
1697,1698,64,38,32,90065,3,0.70,2,0,0,0,0,1,1
340,341,59,34,91,93524,1,2.60,1,0,0,0,0,1,1
1927,1928,35,10,62,93106,3,2.30,1,0,0,0,0,0,0
2330,2331,31,5,72,95133,4,1.80,2,242,0,0,0,1,1
2631,2632,47,20,62,92521,1,2.67,2,0,0,0,0,1,0


In [6]:

attributes = ['Personal Loan','Age','Experience', 'Income','ZIP Code', 'Family',
       'CCAvg', 'Education', 'Mortgage', 'Securities Account', 'CD Account',
       'Online','CreditCard']

In [7]:
#Preparing the training data
train_data = train_data[attributes]
train_data.to_csv('train.csv', index=False, header=False)

In [8]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [9]:
#Creating a Xgboost session
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,role, instance_count=1, instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [10]:
#Model fit
xgb.fit({'train': s3_input_train})

2021-11-15 20:28:37 Starting - Starting the training job...
2021-11-15 20:28:57 Starting - Launching requested ML instancesProfilerReport-1637008116: InProgress
......
2021-11-15 20:30:02 Starting - Preparing the instances for training............
2021-11-15 20:32:06 Downloading - Downloading input data
2021-11-15 20:32:06 Training - Downloading the training image...
2021-11-15 20:32:35 Uploading - Uploading generated training model.Arguments: train
[2021-11-15:20:32:30:INFO] Running standalone xgboost training.
[2021-11-15:20:32:30:INFO] Path /opt/ml/input/data/validation does not exist!
[2021-11-15:20:32:30:INFO] File size need to be processed in the node: 0.12mb. Available memory size in the node: 8347.82mb
[2021-11-15:20:32:30:INFO] Determined delimiter of CSV input is ','
[20:32:30] S3DistributionType set as FullyReplicated
[20:32:30] 3500x12 matrix with 42000 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[20:32:30] src/tree/updater_prune.cc:74

In [11]:
#Deploying the model
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

--------!

In [15]:
test_data = test_data[attributes]

In [16]:
test_data.head()

,Personal Loan,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
2986,1,55,30,153,94305,2,2.8,2,0,1,1,1,0
2237,0,30,5,134,92647,1,0.0,1,0,0,0,1,1
408,0,60,36,89,91745,2,2.8,1,0,0,0,1,0
1197,0,48,23,8,92866,1,0.4,3,0,0,0,0,1
941,1,38,13,129,92093,4,4.4,1,140,0,0,0,0


In [17]:
#Preparing test data
test_data_array = test_data.drop(['Personal Loan'],axis = 1).values 

In [18]:
from sagemaker.serializers import CSVSerializer
#Predicting for the test data
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(1500,)


In [24]:
cm = pd.crosstab(index=test_data['Personal Loan'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No P Loan", "P Loan"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No P Loan", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("P Loan", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 98.5%

Predicted      No P Loan        P Loan
Observed
No P Loan      99% (1351)     4% (5)
P Loan          1% (17)     96% (127) 



In [25]:
from sklearn.metrics import r2_score
print("R2 score : %.2f" % r2_score(test_data['Personal Loan'],predictions_array))

R2 score : 0.86
